# YOLOv10 PCB Defect Detection Model Testing

This notebook tests a trained YOLOv10 model for PCB defect detection and evaluates its accuracy on the dataset.

## Dataset Classes:
- falsecopper
- missinghole  
- mousebite
- opencircuit
- pinhole
- scratch
- shortcircuit
- spur

The trained model weights are stored in `best.pt` and the dataset is in the `DATASET` folder.

## 1. Setup Environment and Install Dependencies

First, we'll install all the necessary packages for YOLOv10 model testing and evaluation.

In [1]:
# Install required packages
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"✗ Failed to install {package}: {e}")

# Install YOLOv10 and dependencies
packages = [
    "ultralytics",
    "torch",
    "torchvision",
    "opencv-python",
    "matplotlib",
    "seaborn",
    "pillow",
    "numpy",
    "pandas",
    "scikit-learn",
    "tqdm"
]

print("Installing required packages...")
for package in packages:
    install_package(package)

print("\n🎉 All packages installed successfully!")

Installing required packages...
✓ Successfully installed ultralytics
✓ Successfully installed torch
✓ Successfully installed torchvision
✓ Successfully installed opencv-python
✓ Successfully installed matplotlib
✓ Successfully installed seaborn
✓ Successfully installed pillow
✓ Successfully installed numpy
✓ Successfully installed pandas
✓ Successfully installed scikit-learn
✓ Successfully installed tqdm

🎉 All packages installed successfully!


## 2. Import Required Libraries

Import all necessary libraries for model testing and evaluation.

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
from PIL import Image
import torch
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('default')
sns.set_palette("husl")

print("📚 All libraries imported successfully!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🖥️  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\ashwi\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
📚 All libraries imported successfully!
🔥 PyTorch version: 2.8.0+cpu
🖥️  CUDA available: False


## 3. Load the Trained YOLOv10 Model

Load the trained model from the `best.pt` file and verify its configuration.

In [3]:
# Define paths
model_path = "best.pt"
dataset_path = "DATASET"
data_yaml_path = os.path.join(dataset_path, "data.yaml")

# Load the trained YOLOv10 model
try:
    model = YOLO(model_path)
    print(f"✅ Model loaded successfully from {model_path}")
    
    # Display model information
    print(f"\n📊 Model Summary:")
    print(f"Model type: {model.model}")
    print(f"Device: {model.device}")
    
    # Load dataset configuration
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    class_names = data_config['names']
    print(f"\n🏷️  Dataset Classes ({len(class_names)}):")
    for idx, class_name in class_names.items():
        print(f"  {idx}: {class_name}")
        
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please make sure 'best.pt' exists in the current directory.")

WARNING best.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolo11n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...

requirements: AutoUpdate success  2.6s
WARNING requirements: Restart runtime or rerun command for updates to take effect

✅ Model loaded successfully from best.pt

📊 Model Summary:
Model type: DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False

## 4. Load and Prepare Test Dataset

Load images from the validation dataset and prepare them for inference.

In [4]:
# Get validation images
valid_images_path = os.path.join(dataset_path, "valid", "images")
valid_labels_path = os.path.join(dataset_path, "valid", "labels")

# Get list of validation images
valid_image_files = []
if os.path.exists(valid_images_path):
    valid_image_files = [f for f in os.listdir(valid_images_path) 
                        if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    print(f"📁 Found {len(valid_image_files)} validation images")
else:
    print(f"⚠️  Validation images path not found: {valid_images_path}")

# Display sample filenames
print(f"\n📋 Sample validation images:")
for i, filename in enumerate(valid_image_files[:10]):
    print(f"  {i+1}: {filename}")
if len(valid_image_files) > 10:
    print(f"  ... and {len(valid_image_files) - 10} more")

# Check if labels exist
if os.path.exists(valid_labels_path):
    valid_label_files = [f for f in os.listdir(valid_labels_path) if f.endswith('.txt')]
    print(f"\n🏷️  Found {len(valid_label_files)} validation label files")
else:
    print(f"⚠️  Validation labels path not found: {valid_labels_path}")
    valid_label_files = []

📁 Found 1908 validation images

📋 Sample validation images:
  1: 01_missing-hole_open-circuit_01_jpg.rf.4c2f7ed779618d760b07858ac3355d47.jpg
  2: 01_missing-hole_open-circuit_01_jpg.rf.4de5e550eccc672422dfb804a3bf98cd.jpg
  3: 01_missing-hole_open-circuit_02_jpg.rf.3f744cc3ff08aa2a45b933db997106d0.jpg
  4: 01_missing-hole_open-circuit_02_jpg.rf.515ee2c885413b4453593c4b1360b0b4.jpg
  5: 01_missing-hole_open-circuit_02_jpg.rf.df2d5f7c35e9c378fcbe7cc6a86d591b.jpg
  6: 01_missing-hole_open-circuit_02_jpg.rf.ee1cb9489af207ce583eb4498a0130fa.jpg
  7: 01_missing-hole_open-circuit_03_jpg.rf.2beb33747b6ca357758952c5a7a1f4ff.jpg
  8: 01_missing-hole_open-circuit_05_jpg.rf.2d70d299d2c37ab2520a84fb1642d012.jpg
  9: 01_missing-hole_open-circuit_05_jpg.rf.fc6ba15668df3a00967abc36285a8f64.jpg
  10: 01_missing-hole_short_10_jpg.rf.3f11f3a1617d9cf3299558c2feb0e3b3.jpg
  ... and 1898 more

🏷️  Found 1908 validation label files


## 5. Run Model Validation

Use the built-in validation function to get comprehensive accuracy metrics.

In [5]:
# Run validation on the dataset
print("🚀 Running model validation...")
print("This may take a few minutes depending on dataset size...\n")

try:
    # Validate the model using the data.yaml file
    results = model.val(data=data_yaml_path, save_json=True, plots=True)
    
    print("✅ Validation completed successfully!\n")
    
    # Extract key metrics
    print("📊 VALIDATION RESULTS:")
    print("=" * 50)
    
    # Overall metrics
    if hasattr(results, 'box'):
        box_results = results.box
        print(f"📈 Overall Performance:")
        print(f"   mAP@0.5:    {box_results.map50:.4f}")
        print(f"   mAP@0.5:0.95: {box_results.map:.4f}")
        print(f"   Precision:  {box_results.mp:.4f}")
        print(f"   Recall:     {box_results.mr:.4f}")
        print(f"   F1-Score:   {2 * (box_results.mp * box_results.mr) / (box_results.mp + box_results.mr):.4f}")
    
    # Class-wise metrics
    if hasattr(results, 'maps'):
        print(f"\n🏷️  Class-wise mAP@0.5:")
        for i, (class_id, class_name) in enumerate(class_names.items()):
            if i < len(results.maps):
                print(f"   {class_name:12}: {results.maps[i]:.4f}")
    
    print(f"\n⏱️  Inference Speed: {results.speed['inference']:.2f} ms per image")
    print(f"📁 Results saved to: runs/detect/val/")
    
except Exception as e:
    print(f"❌ Error during validation: {e}")
    results = None

🚀 Running model validation...
This may take a few minutes depending on dataset size...

Ultralytics 8.3.203  Python-3.13.7 torch-2.8.0+cpu CPU (12th Gen Intel Core i5-12450HX)
YOLOv10n summary (fused): 102 layers, 2,266,728 parameters, 0 gradients, 6.5 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 3.80.6 MB/s, size: 28.2 KB)
val: Scanning C:\Project\PCB\DATASET\valid\labels... 1908 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1908/1908 369.7it/s 5.2s0.1s
val: New cache created: C:\Project\PCB\DATASET\valid\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 120/120 1.0it/s 1:591.0ss
                   all       1908       9908      0.926      0.889       0.95      0.535
           falsecopper        285       1210      0.932      0.863       0.95      0.533
           missinghole        226        982      0.932      0.953      0.976      0.522
             mousebite        326       1433      0.933   

## 6. Visualize Sample Predictions

Run inference on sample validation images and display predictions with bounding boxes.

In [6]:
# Select random sample images for visualization
import random

def visualize_predictions(image_files, num_samples=6):
    """Visualize model predictions on sample images"""
    
    if len(image_files) == 0:
        print("No images found for visualization")
        return
    
    # Select random sample images
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
    
    # Create subplot grid
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    print(f"🖼️  Visualizing predictions on {len(sample_images)} sample images:")
    
    for idx, image_file in enumerate(sample_images):
        if idx >= len(axes):
            break
            
        image_path = os.path.join(valid_images_path, image_file)
        
        try:
            # Run inference
            pred_results = model(image_path, conf=0.25, iou=0.45)
            
            # Get the annotated image
            annotated_image = pred_results[0].plot()
            
            # Convert BGR to RGB for matplotlib
            annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
            
            # Display image
            axes[idx].imshow(annotated_image_rgb)
            axes[idx].set_title(f"{image_file[:20]}..." if len(image_file) > 20 else image_file, 
                               fontsize=10)
            axes[idx].axis('off')
            
            # Print detection details
            detections = pred_results[0].boxes
            if detections is not None and len(detections) > 0:
                print(f"  📍 {image_file}: {len(detections)} detections")
                for i, box in enumerate(detections):
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    class_name = class_names.get(class_id, f"Class_{class_id}")
                    print(f"     - {class_name}: {confidence:.3f}")
            else:
                print(f"  📍 {image_file}: No detections")
                
        except Exception as e:
            print(f"  ❌ Error processing {image_file}: {e}")
            axes[idx].text(0.5, 0.5, f"Error loading\n{image_file}", 
                          transform=axes[idx].transAxes, ha='center', va='center')
            axes[idx].set_title(image_file, fontsize=10)
            axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(len(sample_images), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle('YOLOv10 PCB Defect Detection - Sample Predictions', 
                 fontsize=16, y=1.02)
    plt.show()

# Run visualization
if valid_image_files:
    visualize_predictions(valid_image_files, num_samples=6)
else:
    print("❌ No validation images found for visualization")

🖼️  Visualizing predictions on 6 sample images:

image 1/1 c:\Project\PCB\DATASET\valid\images\01_mouse-bite_short_05_jpg.rf.46170595a5a4e89946c5eb6a7c088982.jpg: 640x640 1 mousebite, 3 shortcircuits, 135.1ms
Speed: 4.8ms preprocess, 135.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
  📍 01_mouse-bite_short_05_jpg.rf.46170595a5a4e89946c5eb6a7c088982.jpg: 4 detections
     - mousebite: 0.676
     - shortcircuit: 0.591
     - shortcircuit: 0.518
     - shortcircuit: 0.491

image 1/1 c:\Project\PCB\DATASET\valid\images\07_spurious-copper_pinhole_04_jpg.rf.914e9edb8d289795404251d8d195da9a.jpg: 640x640 6 falsecoppers, 4 pinholes, 89.2ms
Speed: 5.1ms preprocess, 89.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
  📍 07_spurious-copper_pinhole_04_jpg.rf.914e9edb8d289795404251d8d195da9a.jpg: 10 detections
     - falsecopper: 0.863
     - falsecopper: 0.792
     - pinhole: 0.694
     - falsecopper: 0.688
     - falsecopper: 0.686
     - pinhole: 0.602


<Figure size 1500x1000 with 6 Axes>

## 7. Display Validation Results Charts

Load and display the validation charts generated by YOLO.

In [7]:
# Display validation charts generated by YOLO
def display_validation_charts():
    """Display the validation charts generated by YOLO"""
    
    # Common paths where YOLO saves validation results
    results_paths = [
        "runs/detect/val",
        "runs/detect/val2", 
        "runs/detect/val3"
    ]
    
    # Find the most recent results directory
    results_dir = None
    for path in results_paths:
        if os.path.exists(path):
            results_dir = path
            break
    
    if not results_dir:
        print("❌ No validation results directory found")
        return
    
    print(f"📊 Loading validation charts from: {results_dir}")
    
    # List of chart files to display
    chart_files = [
        "confusion_matrix.png",
        "F1_curve.png", 
        "P_curve.png",
        "R_curve.png",
        "PR_curve.png"
        
    ]
    
    # Display available charts
    available_charts = []
    for chart_file in chart_files:
        chart_path = os.path.join(results_dir, chart_file)
        if os.path.exists(chart_path):
            available_charts.append((chart_file, chart_path))
    
    if not available_charts:
        print("❌ No validation charts found")
        return
    
    # Create subplots
    num_charts = len(available_charts)
    cols = min(2, num_charts)
    rows = (num_charts + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if num_charts == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes.flatten() if cols > 1 else [axes]
    else:
        axes = axes.flatten()
    
    for idx, (chart_name, chart_path) in enumerate(available_charts):
        if idx >= len(axes):
            break
            
        try:
            # Load and display image
            img = plt.imread(chart_path)
            axes[idx].imshow(img)
            axes[idx].set_title(chart_name.replace('.png', '').replace('_', ' ').title(), 
                               fontsize=12)
            axes[idx].axis('off')
            print(f"  ✅ Loaded: {chart_name}")
        except Exception as e:
            print(f"  ❌ Error loading {chart_name}: {e}")
            axes[idx].text(0.5, 0.5, f"Error loading\n{chart_name}", 
                          transform=axes[idx].transAxes, ha='center', va='center')
            axes[idx].set_title(chart_name, fontsize=12)
            axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(len(available_charts), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle('YOLOv10 Validation Results', fontsize=16, y=1.02)
    plt.show()

# Display the charts
display_validation_charts()

📊 Loading validation charts from: runs/detect/val
  ✅ Loaded: confusion_matrix.png


<Figure size 1500x500 with 1 Axes>

## 8. Performance Summary and Model Analysis

Create a comprehensive summary of the model's performance.

In [8]:
# Create comprehensive performance summary
def create_performance_summary():
    """Create a detailed performance summary"""
    
    print("📊 YOLOv10 PCB DEFECT DETECTION MODEL - PERFORMANCE SUMMARY")
    print("=" * 70)
    
    # Model Information
    print(f"\n🤖 MODEL INFORMATION:")
    print(f"   Model Path:      {model_path}")
    print(f"   Model Type:      YOLOv10")
    print(f"   Device:          {model.device}")
    print(f"   Classes:         {len(class_names)}")
    
    # Dataset Information  
    print(f"\n📁 DATASET INFORMATION:")
    print(f"   Dataset Path:    {dataset_path}")
    print(f"   Validation Images: {len(valid_image_files)}")
    print(f"   Label Files:     {len(valid_label_files)}")
    
    # Class Distribution
    print(f"\n🏷️  CLASSES:")
    for class_id, class_name in class_names.items():
        print(f"   {class_id}: {class_name}")
    
    # Performance Metrics (if validation was successful)
    if results is not None:
        print(f"\n📈 PERFORMANCE METRICS:")
        if hasattr(results, 'box'):
            box_results = results.box
            print(f"   mAP@0.5:         {box_results.map50:.4f} ({box_results.map50*100:.2f}%)")
            print(f"   mAP@0.5:0.95:    {box_results.map:.4f} ({box_results.map*100:.2f}%)")
            print(f"   Precision:       {box_results.mp:.4f} ({box_results.mp*100:.2f}%)")
            print(f"   Recall:          {box_results.mr:.4f} ({box_results.mr*100:.2f}%)")
            
            f1_score = 2 * (box_results.mp * box_results.mr) / (box_results.mp + box_results.mr)
            print(f"   F1-Score:        {f1_score:.4f} ({f1_score*100:.2f}%)")
        
        print(f"\n⏱️  PERFORMANCE:")
        print(f"   Inference Speed: {results.speed['inference']:.2f} ms/image")
        print(f"   FPS:             {1000/results.speed['inference']:.1f}")
        
        # Model assessment
        print(f"\n🎯 MODEL ASSESSMENT:")
        if hasattr(results, 'box'):
            map50 = box_results.map50
            if map50 >= 0.9:
                assessment = "🟢 EXCELLENT - Very high accuracy"
            elif map50 >= 0.8:
                assessment = "🟡 GOOD - High accuracy, suitable for production"
            elif map50 >= 0.6:
                assessment = "🟠 FAIR - Moderate accuracy, may need improvement"
            else:
                assessment = "🔴 POOR - Low accuracy, requires significant improvement"
            
            print(f"   Overall Rating:  {assessment}")
            
            # Per-class performance
            if hasattr(results, 'maps') and len(results.maps) > 0:
                print(f"\n📊 CLASS-WISE PERFORMANCE:")
                class_performance = []
                for i, (class_id, class_name) in enumerate(class_names.items()):
                    if i < len(results.maps):
                        map_score = results.maps[i]
                        class_performance.append((class_name, map_score))
                        status = "🟢" if map_score >= 0.8 else "🟡" if map_score >= 0.6 else "🔴"
                        print(f"   {status} {class_name:12}: {map_score:.4f} ({map_score*100:.1f}%)")
                
                # Find best and worst performing classes
                if class_performance:
                    best_class = max(class_performance, key=lambda x: x[1])
                    worst_class = min(class_performance, key=lambda x: x[1])
                    print(f"\n   🏆 Best:  {best_class[0]} ({best_class[1]:.4f})")
                    print(f"   ⚠️  Worst: {worst_class[0]} ({worst_class[1]:.4f})")
    else:
        print(f"\n❌ Validation metrics not available")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    if results is not None and hasattr(results, 'box'):
        map50 = box_results.map50
        precision = box_results.mp
        recall = box_results.mr
        
        if map50 < 0.8:
            print(f"   • Consider training for more epochs")
            print(f"   • Try data augmentation techniques")
            print(f"   • Verify label quality and consistency")
        
        if precision < 0.8:
            print(f"   • Too many false positives - consider increasing confidence threshold")
        
        if recall < 0.8:
            print(f"   • Missing detections - consider lowering confidence threshold")
            print(f"   • Add more training data for underrepresented classes")
        
        if map50 >= 0.8:
            print(f"   • ✅ Model performance is good for deployment")
            print(f"   • Consider fine-tuning confidence thresholds for production")
    else:
        print(f"   • Run validation to get detailed recommendations")
    
    print(f"\n" + "=" * 70)

# Generate the summary
create_performance_summary()

📊 YOLOv10 PCB DEFECT DETECTION MODEL - PERFORMANCE SUMMARY

🤖 MODEL INFORMATION:
   Model Path:      best.pt
   Model Type:      YOLOv10
   Device:          cpu
   Classes:         8

📁 DATASET INFORMATION:
   Dataset Path:    DATASET
   Validation Images: 1908
   Label Files:     1908

🏷️  CLASSES:
   0: falsecopper
   1: missinghole
   2: mousebite
   3: opencircuit
   4: pinhole
   5: scratch
   6: shortcircuit
   7: spur

📈 PERFORMANCE METRICS:
   mAP@0.5:         0.9497 (94.97%)
   mAP@0.5:0.95:    0.5354 (53.54%)
   Precision:       0.9262 (92.62%)
   Recall:          0.8889 (88.89%)
   F1-Score:        0.9071 (90.71%)

⏱️  PERFORMANCE:
   Inference Speed: 53.34 ms/image
   FPS:             18.7

🎯 MODEL ASSESSMENT:
   Overall Rating:  🟢 EXCELLENT - Very high accuracy

📊 CLASS-WISE PERFORMANCE:
   🔴 falsecopper : 0.5326 (53.3%)
   🔴 missinghole : 0.5224 (52.2%)
   🔴 mousebite   : 0.5290 (52.9%)
   🔴 opencircuit : 0.4553 (45.5%)
   🔴 pinhole     : 0.4401 (44.0%)
   🟢 scratch     :

## 9. Test on Individual Images (Optional)

You can test the model on individual images by running the cell below with your own image paths.

In [ ]:
# Test on individual images
def test_individual_image(image_path, conf_threshold=0.25, iou_threshold=0.45):
    """Test model on a single image"""
    
    if not os.path.exists(image_path):
        print(f"❌ Image not found: {image_path}")
        return
    
    try:
        # Run inference
        results = model(image_path, conf=conf_threshold, iou=iou_threshold)
        
        # Get the annotated image
        annotated_image = results[0].plot()
        annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        
        # Display results
        plt.figure(figsize=(12, 8))
        plt.imshow(annotated_image_rgb)
        plt.title(f"YOLOv10 Predictions - {os.path.basename(image_path)}", fontsize=14)
        plt.axis('off')
        plt.show()
        
        # Print detection details
        detections = results[0].boxes
        if detections is not None and len(detections) > 0:
            print(f"\n🎯 Found {len(detections)} detections:")
            for i, box in enumerate(detections):
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = class_names.get(class_id, f"Class_{class_id}")
                
                # Get bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                
                print(f"   {i+1}. {class_name}")
                print(f"      Confidence: {confidence:.3f}")
                print(f"      Box: ({x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f})")
        else:
            print("🔍 No detections found")
            
    except Exception as e:
        print(f"❌ Error processing image: {e}")

# Example usage - uncomment and modify the path to test on your own images
# test_individual_image("path/to/your/test/image.jpg")

print("💡 To test on individual images:")
print("   1. Uncomment the last line above")
print("   2. Replace the path with your image path") 
print("   3. Run the cell")
print("\n📁 Available validation images:")
for i, img in enumerate(valid_image_files[:5]):
    img_path = os.path.join(valid_images_path, img)
    print(f"   test_individual_image(r'{img_path}')")
if len(valid_image_files) > 5:
    print(f"   ... and {len(valid_image_files) - 5} more")